# Tutorial 4: Hybrid sampling
In this tutorial, we will be introduced to the concept of *hybrid sampling*; the process of using an emulator as an additional prior in a Bayesian analysis.
Hybrid sampling can be used to massively speed up parameter estimation algorithms based on MCMC and Bayesian, by utilizing all the information captured by the emulator.
It is assumed here that the reader has successfully completed the first tutorial ([Basic usage](1_basic_usage.ipynb)) and has a basic knowledge of how to perform a Bayesian parameter estimation in Python.
This tutorial mostly covers what can be found in the section on [Hybrid sampling](https://prism-tool.readthedocs.io/en/latest/user/using_prism.html#hybrid-sampling) in the online documentation, but in a more interactive way.

A common problem when using MCMC methods is that it can often take a very long time for MCMC to find its way on the posterior probability distribution function (PDF), which is often referred to as the *burn-in phase*.
This is because, when considering a parameter set for an MCMC walker (where every walker creates its own Markov chain), there is usually no prior information that this parameter set is (un)likely to result into a desirable model realization.
This means that such a parameter set must first be evaluated in the model before any probabilities can be calculated.
However, by constructing an emulator of the model, we can use it as an additional prior for the posterior probability calculation.
Therefore, although *PRISM* is primarily designed to make analyzing models much more efficient and accessible than normal MCMC methods, it is also very capable of enhancing them.
This process is called *hybrid sampling*, which we can perform easily with the `prism.utils` module and which we will explain/explore below.

## Algorithm
Hybrid sampling allows us to use *PRISM* to first analyze a model’s behavior, and later use the gathered information to speed up parameter estimations (by using the emulator as an additional prior in a Bayesian analysis).
Hybrid sampling works in the following way:

1. Whenever an MCMC walker proposes a new sample, it is first passed to the emulator of the model;
2. If the sample is not within the defined parameter space, it automatically receives a prior probability of zero (or $-\infty$ in case of logarithmic probabilities).
   Else, it will be evaluated in the emulator;
3. If the sample is labeled as implausible by the emulator, it also receives a prior probability of zero.
   If it is plausible, the sample is evaluated in the same way as for normal sampling;
4. Optionally, a scaled value of the first implausibility cut-off is used as an exploratory method by adding an additional (non-zero) prior probability.
   This can be enabled by using the *impl_prior* input argument in the `get_hybrid_lnpost_fn()`-function factory (which we will cover in a bit).

Since the emulator that *PRISM* makes of a model is not defined outside of the parameter space given by `modellink_obj.par_rng`, the second step is necessary to make sure the results are valid.
There are several advantages of using hybrid sampling over normal sampling:

- Acceptable samples are guaranteed to be within plausible space;
- This in turn makes sure that the model is only evaluated for plausible samples, which heavily reduces the number of required evaluations;
- No burn-in phase is required, as the starting positions of the MCMC walkers are chosen to be in plausible space;
- As a consequence, varying the number of walkers tends to have a much lower negative impact on the convergence probability and speed;
- Samples with low implausibility values can optionally be favored.

## Usage
### Preparation
Before we can get started, let's import all the basic packages and definitions that we need to do hybrid sampling:

In [ ]:
# Imports
from corner import corner
from e13tools.pyplot import f2tex
from e13tools.sampling import lhd
import matplotlib.pyplot as plt
import numpy as np
from prism import Pipeline
from prism.modellink import GaussianLink
from prism.utils import get_hybrid_lnpost_fn, get_walkers

We also require a constructed emulator of the desired model.
For this tutorial, we will use the trusty `GaussianLink` class again, but feel free to use a different `ModelLink` subclass or modify the settings below (keep in mind that the second iteration may take a little while to analyze):

In [ ]:
# Emulator construction
# Set required emulator iteration
emul_i = 2

# Create GaussianLink object
model_data = {3: [3.0, 0.1],    # f(3) = 3.0 +- 0.1
              5: [5.0, 0.1],    # f(5) = 5.0 +- 0.1
              7: [3.0, 0.1]}    # f(7) = 3.0 +- 0.1
modellink_obj = GaussianLink(model_data=model_data)

# Initialize Pipeline
pipe = Pipeline(modellink_obj, working_dir='prism_hybrid')

# Construct required iterations
for i in range(pipe.emulator.emul_i+1, emul_i+1):
    # For reproducibility, set the NumPy random seed for each iteration
    np.random.seed(1313*i)

    # Use analyze=False to use different impl_cuts for different iterations
    pipe.construct(i, analyze=False)

    # Use different analysis settings for iteration 2
    if(i == 2):
        pipe.base_eval_sam = 8000
        pipe.impl_cut = [4.0, 3.8, 3.6]

    # Analyze iteration
    pipe.analyze()

# Set NumPy random seed to something random
np.random.seed()

### Implementing hybrid sampling
In order to help us with combining *PRISM* with MCMC to use hybrid sampling, the `prism.utils` module provides two support functions: `get_walkers()` and `get_hybrid_lnpost_fn()`.
As these functions will make our job of using hybrid sampling much easier, let's take a look at both of them.

#### get_walkers()
If we were to do normal MCMC sampling, then it would be required for us to provide our sampling script (whatever that may be) with the starting positions of all MCMC walkers, or at least with the number of MCMC walkers we want.
Normally, we would have no idea what starting positions to choose, as we know nothing about the corresponding model parameter space.
However, as we have already constructed an emulator of the target model, we actually do have some information at our disposal that could help us in choosing their starting positions.
Preferably, we would choose the starting positions of the MCMC walkers in the plausible space of the emulator, as we know that these will be close to the desired parameter set.

This is where the `get_walkers()`-function comes in.
It allows us to obtain a set of plausible starting positions for our MCMC walkers, given a `Pipeline` object.
By default, `get_walkers()` returns the available plausible samples of the last constructed iteration (`pipe.impl_sam`), but we can also supply it with an integer stating how many starting positions we want to propose or we can provide our own set of proposed starting positions.
Below we can see these three different scenarios in action:

In [ ]:
# Use impl_sam if it is available (default)
n_walkers, p0_walkers = get_walkers(pipe)
print("Number of plausible starting positions (default) : %i" % (n_walkers))

# Propose 10000 starting positions
init_walkers = 10000
n_walkers, p0_walkers = get_walkers(pipe, init_walkers=init_walkers)
print("Number of plausible starting positions (integer) : %i" % (n_walkers))

# Propose custom set of 10000 starting positions
init_walkers = lhd(10000, modellink_obj._n_par, modellink_obj._par_rng)
n_walkers, p0_walkers = get_walkers(pipe, init_walkers=init_walkers)
print("Number of plausible starting positions (custom)  : %i" % (n_walkers), flush=True)

# Request at least 1000 plausible starting positions (requires v1.1.4 or later)
n_walkers, p0_walkers = get_walkers(pipe, req_n_walkers=1000)
print("Number of plausible starting positions (specific): %i" % (n_walkers))

Note that when there is still a large part of parameter space left, the number of plausible starting positions will probably greatly vary between runs when a specific number is requested.
As hybrid sampling works better when plausible space is very small, it is usually recommended that we first construct a good emulator before attempting to do hybrid sampling.
This is also why we used different implausibility parameters for the analysis of the second emulator iteration.

As *PRISM*'s sampling methods operate in parameter space, `get_walkers()` automatically assumes that all starting positions are defined in parameter space.
However, as some sampling methods use unit space, we can request normalized starting positions by providing `get_walkers()` with *unit_space=True*.
We have to keep in mind though that, because of the way the emulator works, there is no guarantee for a specific number of plausible starting positions to be returned.
Having the desired emulator iteration already analyzed may give us an indication of how many starting positions in total we need to propose to be left with a specific number.
However, starting with v1.1.4, the `get_walkers()`-function takes the *req_n_walkers* argument, which allows us to request a specific minimum number of plausible starting positions (as shown).

When the starting positions of our MCMC walkers have been determined, we can use them in our MCMC sampling script, avoiding the previously mentioned burn-in phase.
Although this in itself can already be very useful, it does not allow us to perform hybrid sampling yet.
In order to do this, we additionally need something else.

#### get_hybrid_lnpost_fn()
Most MCMC methods require the definition of an `lnpost()`-function.
This function takes a proposed sample step in the chain of an MCMC walker and returns the corresponding natural logarithm of the posterior probability.
The returned value is then compared with the current value, and the new step is accepted with some probability based on this comparison.
As we have learned, in order to do hybrid sampling, we require the [algorithm](#Algorithm) described before.
Therefore, we need to modify this `lnpost()`-function to first evaluate the proposed sample in the emulator and only perform the normal posterior probability calculation when this sample is plausible.

Making this modification is the job of the `get_hybrid_lnpost_fn()`-function factory.
It takes a user-defined `lnpost()`-function (as input argument *lnpost_fn*) and a `Pipeline` object, and returns a function definition ``hybrid_lnpost(par_set, *args, **kwargs)``.
This `hybrid_lnpost()`-function first analyzes a proposed *par_set* in the emulator, and passes *par_set* (along with any additional arguments) to `lnpost()` if the sample is plausible, or returns $-\infty$ if it is not.
The return-value of the `lnpost()`-function is then returned by the `hybrid_lnpost()`-function as well.

The reason why we use a function factory here, is that it allows us to validate all input arguments once and then save them as local variables for the `hybrid_lnpost()`-function.
Not only does this avoid that we have to provide and validate all input arguments (like *emul_i* and *pipeline_obj*) for every individual proposal, but it also ensures that we always use the same arguments, as we cannot modify the local variables of a function.

So, let's define an `lnpost()`-function, which uses a simple Gaussian probability function:

In [ ]:
# Pre-calculate the data variance. Assume data_err is centered
data_var = np.array([err[0]**2 for err in modellink_obj._data_err])

# Add a global counter to measure the number of times the model is called
global call_counter
call_counter = 0


# Define lnpost
def lnpost(par_set):
    # Check if par_set is within parameter space and return -infty if not
    # Note that this is not needed if we solely use lnpost for hybrid sampling
    par_rng = modellink_obj._par_rng
    if not ((par_rng[:, 0] <= par_set)*(par_set <= par_rng[:, 1])).all():
        return(-np.infty)

    # Increment call_counter by one
    global call_counter
    call_counter += 1

    # Convert par_set to par_dict
    par_dict = dict(zip(modellink_obj._par_name, par_set))

    # Evaluate model at requested parameter set
    mod_out = modellink_obj.call_model(emul_i, par_dict,
                                       modellink_obj._data_idx)

    # Get model and total variances
    md_var = modellink_obj.get_md_var(emul_i, par_dict,
                                      modellink_obj._data_idx)
    tot_var = md_var+data_var

    # Calculate the posterior probability
    return(-0.5*(np.sum((modellink_obj._data_val-mod_out)**2/tot_var)))

As we already have a way of evaluating our model using the `ModelLink` subclass, we used this to our advantage and simply made the `call_model()` and `get_md_var()`-methods part of the posterior probability calculation.
We also know that the data variance will not change in between evaluations, so we calculated it once outside of the function definition.
Keep in mind that hybrid sampling itself already checks if a proposed sample is within parameter space, and it was therefore not necessary to check for this in our `lnpost()`-function, unless we are going to do normal sampling as well (in which case it acts as a prior).

Now that we have defined our `lnpost()`-function, we can create a specialized version of it that automatically performs hybrid sampling:

In [ ]:
hybrid_lnpost = get_hybrid_lnpost_fn(lnpost, pipe)

As with the `get_walkers()`-function, the `get_hybrid_lnpost_fn()`-function factory can be set to perform in unit space by providing it with *unit_space=True*.
This will make the returned `hybrid_lnpost()`-function expect normalized samples.
We also have to keep in mind that by calling the `get_hybrid_lnpost_fn()`-function factory, *PRISM* has turned off all normal logging.
This is to avoid having thousands of similar logging messages being made.
It can be turned back on again by executing `pipe.do_logging = True`.

We can check if the returned `hybrid_lnpost()`-function really will perform hybrid sampling, by evaluating an implausible sample in it (in almost all cases, `[1, 1, 1]` will be implausible for the `GaussianLink` class. If not, feel free to change the sample):

In [ ]:
# Define a sample
par_set = [1, 1, 1]

# Evaluate it in the pipeline to check if it is plausible
pipe.evaluate(par_set)

# Evaluate this sample in both lnpost and hybrid_lnpost
print()
print("       lnpost(%s) = %s" % (par_set, lnpost(par_set)))
print("hybrid_lnpost(%s) = %s" % (par_set, hybrid_lnpost(par_set)))

Here, we can see that while the proposed sample does give a finite log-posterior probability when evaluated in the `lnpost()`-function, this is not the case when evaluated in the `hybrid_lnpost()`-function due to the sample not being plausible in the emulator.

And, finally, as it is very likely that we will frequently use `get_walkers()` and `get_hybrid_lnpost_fn()` together, the `get_walkers()`-function allows for the *lnpost_fn* input argument to be provided to it.
Doing so will automatically call the `get_hybrid_lnpost_fn()`-function factory using the provided *lnpost_fn* and the same input arguments given to `get_walkers()`, and return the obtained `hybrid_lnpost()`-function in addition to the starting positions of the MCMC walkers.
So, before we get into applying hybrid sampling to our model, let's use the `get_walkers()`-function to obtain all the variables we need:

In [ ]:
n_walkers, p0_walkers, hybrid_lnpost = get_walkers(pipe, lnpost_fn=lnpost)

## Application
Now that we know how to apply hybrid sampling to our model in theory, let's see it in action.
When picking an MCMC sampling method to use, we have to keep in mind that it must allow for the starting positions of the MCMC walkers to be provided; and it must be able to use a custom log-posterior probability function.
As this is very common for most sampling methods, we will cover a few of the most popular MCMC sampling packages in Python in separate sections.

Because the emulator can be used in two different ways to speed up the parameter estimation process, we will explore three different samplers for each MCMC sampling package: *normal* (solely use normal sampling); *kick-started* (use normal sampling but start in plausible space); and *hybrid* (use normal sampling with added emulator prior).
By using these three different samplers, we can explore what the effect is of introducing an emulator to the MCMC sampling process.
So, for consistency, let's define their names here:

In [ ]:
names = ["Normal", "Kick-Started", "Hybrid"]

Before continuing, please make sure that the appropriate package is installed before going to the related section.
Also, if you would like to see an example of a different sampling package in here, please open a [GitHub issue](https://github.com/1313e/PRISM/issues) about it (keep in mind that all requested packages must be `pip`-installable).

### emcee
Probably one of the most popular MCMC sampling packages in Python for quite some time now, is the [*emcee*](https://emcee.readthedocs.io/en/latest/) package.
For that reason, it would be weird to not include it here.
So, let's import the required definitions and see how we can use *emcee* to do hybrid sampling (we also call `get_walkers()` again to make sure any changes made previously are overridden):

In [ ]:
from emcee import EnsembleSampler
n_walkers, p0_walkers, hybrid_lnpost = get_walkers(pipe, lnpost_fn=lnpost)

Now that we have the required class for doing the sampling, we can set up the three different samples.
As the `EnsembleSampler` requires that the number of MCMC walkers is even, we will also have to make sure that this is the case (by duplicating all walkers if it is not):

In [ ]:
# Duplicate all MCMC walkers if n_walkers is odd
if n_walkers % 2:
    n_walkers *= 2
    p0_walkers = np.concatenate([p0_walkers]*2)

# Generate some initial positions for the normal sampler
p0_walkers_n = lhd(n_sam=n_walkers,
                   n_val=modellink_obj._n_par,
                   val_rng=modellink_obj._par_rng)

# Create empty dict to hold the samplers
samplers = {}

# Define the three different samplers
# Add their initial positions and their initial model call counter
# Normal sampler
samplers[names[0]] = [p0_walkers_n, 0,
                      EnsembleSampler(nwalkers=n_walkers,
                                      dim=modellink_obj._n_par,
                                      lnpostfn=lnpost)]

# Kick-started sampler
samplers[names[1]] = [p0_walkers, sum(pipe.emulator.n_sam[1:]),
                      EnsembleSampler(nwalkers=n_walkers,
                                      dim=modellink_obj._n_par,
                                      lnpostfn=lnpost)]

# Hybrid sampler
samplers[names[2]] = [p0_walkers, sum(pipe.emulator.n_sam[1:]),
                      EnsembleSampler(nwalkers=n_walkers,
                                      dim=modellink_obj._n_par,
                                      lnpostfn=hybrid_lnpost)]

In the declaration of the three different entries for the `samplers` dict, we can easily see what the differences are between the samplers.
But, everything is now ready to do some (hybrid) sampling:

In [ ]:
# Define number of MCMC iterations
n_iter = 50

# Loop over all samplers
for name, (p0, call_counter, sampler) in samplers.items():
    # Run the sampler for n_iter iterations
    sampler.run_mcmc(p0, n_iter)

    # Make sure to save current values of p0 and call_counter for reruns
    samplers[name][0] = sampler.chain[:, -1]
    samplers[name][1] = call_counter

    # Create a corner plot showing the results
    fig = corner(xs=sampler.flatchain,
                 labels=modellink_obj._par_name,
                 show_titles=True,
                 truths=modellink_obj._par_est)
    fig.suptitle("%s: $%s$ iterations, $%s$ evaluations, $%s$%% acceptance"
                 % (name, f2tex(sampler.iterations), f2tex(call_counter),
                    f2tex(np.average(sampler.acceptance_fraction*100))),
                 y=0.025)
    plt.show()

From these three corner plots, we can learn a few things.
First of all, we can see that the parameter estimation accuracy of the normal sampler (first plot) is noticeably lower than that of the other two.
However, it also used less model evaluations (as shown by the title at the bottom of each plot).
The reason for this is that we added the initial number of model evaluations required for constructing the emulator as well, which is:

In [ ]:
sum(pipe.emulator.n_sam[1:])

If we take this into account, then we see that the number of model evaluations used during the sampling process, is very similar for all three.
However, if we were to run the normal sampler for another *n_iter* iterations, this might change:

In [ ]:
# Get variables for normal sampler
name = names[0]
p0, call_counter, sampler = samplers[name]

# Run the sampler for n_iter iterations
sampler.run_mcmc(p0, n_iter)

# Make sure to save current values of p0 and call_counter for reruns
samplers[name][0] = sampler.chain[:, -1]
samplers[name][1] = call_counter

# Create a corner plot showing the results
fig = corner(xs=sampler.flatchain,
             labels=modellink_obj._par_name,
             show_titles=True,
             truths=modellink_obj._par_est)
fig.suptitle("%s: $%s$ iterations, $%s$ evaluations, $%s$%% acceptance"
             % (name, f2tex(sampler.iterations), f2tex(call_counter),
                f2tex(np.average(sampler.acceptance_fraction*100))),
             y=0.025)
plt.show()

We can see here that the normal sampler has still not reached the same accuracy as the kick-started and hybrid samplers, even though now it used more model evaluations.
This shows how much of an impact skipping the burn-in phase has on the convergence speed of the sampling process, even when using a simple model like our single Gaussian.

Something that we can also note is that the kick-started and hybrid samplers have very, very similar results and accuracies.
This, again, shows the impact of skipping the burn-in phase.
However, it does seem that the hybrid sampler used less evaluations than the kick-started sampler, which is a result of the emulator steering the first into the right direction.
The effect for this simple model is very minor though, but can easily become much more important for more complex models.

### dynesty
Coming soon...